In [ ]:
# ============================================================
# 🤖 SINGLE-AGENT RAG ASSISTANT (LangChain + LlamaIndex)
# Author: James Docktor
# ============================================================

# 🧩 SECTION 1. ENVIRONMENT SETUP
import os
from dotenv import load_dotenv
load_dotenv()

print("✅ Environment loaded successfully.")
print("OPENAI_API_KEY found:", bool(os.getenv("OPENAI_API_KEY")))

# ============================================================
# 🧠 SECTION 2. CORE LIBRARIES
# ============================================================
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

import chromadb

print("✅ Core libraries imported successfully.")

# ============================================================
# 📚 SECTION 3. BUILD LLAMAINDEX RETRIEVER
# ============================================================

# Prepare Chroma vector store
persist_dir = "chroma_db"
os.makedirs(persist_dir, exist_ok=True)
vector_store = ChromaVectorStore(persist_dir=persist_dir)

# Load local docs
docs = SimpleDirectoryReader("docs").load_data()

# Create embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create and persist index
index = VectorStoreIndex.from_documents(docs, embedding=embed_model, vector_store=vector_store)
query_engine = index.as_query_engine(similarity_top_k=3)

def rag_tool_fn(query: str) -> str:
    """Retrieve factual info from local docs."""
    return query_engine.query(query).response

print("✅ LlamaIndex retriever built and ready.")

# ============================================================
# ⚙️ SECTION 4. CREATE LANGCHAIN AGENT
# ============================================================

# Define tools
tools = [
    Tool(
        name="RAG Retriever",
        func=rag_tool_fn,
        description="Use this tool to look up factual info in local documents."
    )
]

# Initialize LLM
llm = OpenAI(temperature=0.4)

# Memory for chat context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    verbose=True,
    memory=memory
)

print("✅ LangChain agent initialized.")

# ============================================================
# 💬 SECTION 5. TEST AGENT REASONING
# ============================================================

agent.run("Explain what retrieval-augmented generation is and why it's useful.")

# ============================================================
# 📊 SECTION 6. EVALUATION (TruLens / Ragas placeholder)
# ============================================================

print("📊 Placeholder: evaluation hooks for TruLens or Ragas go here.")
